In [1]:
import pandas as pd
import numpy as np
import os
import random
import heapq
import time

In [23]:
# Set Working Directory
os.getcwd()
path = "C:/Users/Dennis/OneDrive/Dokumente/03_Master BAOR/05_Kurse/01_Business Analytics/04_Data Analytics Challenge/05_Scripts"
os.chdir(path)

%cd "C:\Users\Dennis\OneDrive\Dokumente\03_Master BAOR\05_Kurse\01_Business Analytics\04_Data Analytics Challenge\05_Scripts"
ccdata = pd.read_csv('creditcard.csv')
#ccdata = ccdata.iloc[:100000,:]
ccdata.Class.value_counts()

c:\Users\Dennis\OneDrive\Dokumente\03_Master BAOR\05_Kurse\01_Business Analytics\04_Data Analytics Challenge\05_Scripts


0    284315
1       492
Name: Class, dtype: int64

In [24]:
def Euclidean_Metric(a,b):
      dis=0
      A = np.array(a)
      B = np.array(b)
      n=A.shape[0]
      for i in range(n):
          dis=dis+(A[i]-B[i])*(A[i]-B[i])
      dis=np.sqrt(dis)
      return dis

samples = ccdata

time_start=time.time()
g_index=0
wrg=0
samples_X=samples.iloc[:,0:-1] #features
samples_Y=samples.iloc[:,-1] #target variable
Minority_sample=samples[samples['Class'] == 1] #minority set
Minority_sample_X=Minority_sample.iloc[:,0:-1] #features of minority set

In [25]:
Minority_X=np.array(Minority_sample_X) #features of Min set
All_X=np.array(samples_X) #features of all samples
n1=All_X.shape[0]-2*Minority_X.shape[0] #Anzahl aller samples - 2 * Anzahl Min samples #284807 - 2* 492 = 283823
print(n1)

# n=int((All_X.shape[0]-2*Minority_X.shape[0])/Minority_X.shape[0])
# print(n)

283823


In [26]:
# Algorithm 1: Noise filtering
dis_matrix=np.zeros((Minority_X.shape[0],All_X.shape[0]),dtype=float) #leere Matrix (492x284807) nur trainingsdatenset

for i in range(0,Minority_X.shape[0]):
    for j in range(0,All_X.shape[0]):
        dis_matrix[i,j]=Euclidean_Metric(Minority_X[i,:],All_X[j,:]) #berechne für jede Min instance die Eucl. distance zu allen sampels im Datensatz
        if(dis_matrix[i,j]==0): #Eucl. distance zu sich selbst
            dis_matrix[i,j]=999999 #Min instance darf nicht sein eigener nächster Nachbar sein
dis_matrix=dis_matrix.tolist()

In [27]:
d=[] #list for noise Min instances
#print(Minority_X.shape[0])
for i in range(Minority_X.shape[0]):
    min_index=list(map(dis_matrix[i].index, heapq.nsmallest(1, dis_matrix[i]))) #index des nächsten Nachbarn in Zeile der aktuellen Min instance
    #print(min_index)
    if(samples_Y[min_index[0]]==0): #nearest neighbour ist aus Maj class -> Noise
        d.append(i)
Minority_X=np.delete(Minority_X,d,axis=0) #lösche alle Noise samples aus Min set
#print(Minority_X.shape)
n=int((n1)/Minority_X.shape[0])
#print(n)

In [28]:
len(Minority_X)

93

In [29]:
# Algorithm 2: Adaptive neighbor instances selection
n = 10 #number of synthetic new instances per qualified Min instance
k = 5
synthetic = np.zeros(((Minority_X.shape[0])*n,Minority_X.shape[1]),dtype=float) #Matrix(qualified Min instances * n x Number of Features)

#print(Minority_X.shape[0])
for i in range(Minority_X.shape[0]):
    min_index=list(map(dis_matrix[i].index, heapq.nsmallest(k, dis_matrix[i]))) #indizes der k nächsten Nachbarn in Zeile der aktuellen qualif. Min instance
    best_index={}
    best_f=0
    for h in range(len(min_index)):
        
        if(samples_Y[min_index[h]]==0): #if h. nearest neighbour belongs to Maj class set -> STOP
            best_index[best_f]=min_index[h]
            best_f+=1
            break
        else:
            best_index[best_f]=min_index[h] #else append h. nearest neighbour to set of qualified nearest neighbours for qualified Min instance
            best_f+=1
    #print(best_index)

    # Algorithm 3: Procedure of ASN-SMOTE (Create new synthetic minority samples)
    for j in range(0,n):
        nn=random.randint(0,len(best_index)-1) #Zufälliges Auswählen eines besten Nachbarn aus dem Set der qualifizierten Nachbarn
        #print(min_index[nn])
        dif=All_X[best_index[nn]]-Minority_X[i] #Differenz des zufällig ausgewählten Nachbarn und der Min instance
        #print(dif)
        gap=random.random() #Wert zw. 0 und 1
        synthetic[g_index]=Minority_X[i]+gap*dif #Generierung einer neuen Min instance
        g_index+=1

In [31]:
#print(synthetic.shape)
#print(wrg)

# synthetic=synthetic[0:synthetic.shape[0]-,:]
labels=np.ones(synthetic.shape[0]) #Kennzeichnung der neuen synthetischen samples als Min instance mit class=1
synthetic=np.insert(synthetic,synthetic.shape[1],values=labels,axis=1)
examples=np.concatenate((samples,synthetic),axis=0) #Verbindung der samples mit den synthetischen samples
time_end=time.time()
del(dis_matrix)

In [32]:
len(examples) - len(All_X)

930